# 1. Spatial-Only Heatmap

We’ll visualize the spatialHeatMap to show how many ships intersect each spatial grid cell.

In [ ]:
import psycopg as pg
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap


# Connect to the PostgreSQL/MobilityDB database
connection = pg.connect(
    host="localhost",
    port="5432",
    dbname="aiscompression",  # Replace with your actual database
    user="postgres",
    password="bankmisr1420"
)

# Query for the spatial heatmap
query = """
    SELECT ST_X(ST_TRANSFORM(cell_geom, 4326)) AS lon, ST_Y(ST_TRANSFORM(cell_geom, 4326)) AS lat, count
    FROM spatialHeatMap;
"""
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

# Apply log transformation to the 'cnt' (count) column to better visualize the data
df['log_count'] = np.log(df['count'] + 1)  # Adding 1 to avoid log(0) issue

# Create a base map centered on Denmark using Folium
m = folium.Map(location=[55.6761, 12.5683], zoom_start=7, tiles="cartodb positron")

# Create a list of heatmap data with latitude, longitude, and log_cnt
heat_data = [[row['lat'], row['lon'], row['log_count']] for index, row in df.iterrows()]

# Create a heatmap layer with red gradient and add it to the map
HeatMap(
    heat_data,
    min_opacity=0.4,  # Minimum transparency for low-count areas
    max_opacity=0.9,  # Maximum opacity for high-count areas
    radius=5,  # Radius of each point
    blur=5,    # Blur effect to smooth the heatmap
    gradient={0.4: 'white', 0.5: 'orange', 0.7: 'red'}  # Custom color gradient
).add_to(m)

# Save and display the map
m.save('SpatialHeatmap.html')
m


# 2. Temporal-Only Heatmap

Next, we’ll visualize the timeHeatMap table to show the distribution of ship counts over time (hours).


In [ ]:
import psycopg as pg
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Connect to the PostgreSQL/MobilityDB database
connection = pg.connect(
    host="localhost",  # or your database host
    port="5432",
    dbname="mobility_analytics_chapter",
    user="postgres",
    password="password"  # Replace with your actual password
)

# Query for the temporal heatmap
query = """
    SELECT cell_t, count
    FROM timeHeatMap;
"""
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()


# df['cell_t'] is in datetime format, trim it to show only the hour
df['cell_t'] = pd.to_datetime(df['cell_t']).dt.strftime('%H:00')  # Format to show only hour (e.g., '15:00')


# Create a heatmap using Seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(df.pivot_table(index='cell_t', values='count', fill_value=0), cmap='Blues', annot=True, fmt='.0f', cbar=False)
# Set custom font properties for labels and title
font_properties = {'family': 'Liberation Serif', 'size': 18, 'color': 'black'}

plt.xlabel('Count of Ships', fontdict=font_properties)
plt.ylabel('Hour of Day', fontdict=font_properties)


# Save the heatmap as a PDF with minimal margins
plt.savefig("TemporalHeatmap.pdf", format="pdf", bbox_inches='tight')

# Show the plot in an interactive window
plt.show()



# 3. Space-Time 3D Heatmap

We’ll visualize the full spatiotemporal data from tHeatmap in 3D using Plotly.

In [ ]:
import psycopg as pg
import pandas as pd
import plotly.express as px


# Connect to the PostgreSQL/MobilityDB database
connection = pg.connect(
    host="localhost",  # or your database host
    port="5432",
    dbname="mobility_analytics_chapter",
    user="postgres",
    password="password"  # Replace with your actual password
)

# Query for the 3D space-time heatmap
query = """
    SELECT ST_X(cell_geom) AS lon, ST_Y(cell_geom) AS lat, cell_t, COUNT(*) AS ship_count
    FROM tHeatmap
    GROUP BY cell_geom, cell_t;
"""
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()
    

# Plot the 3D space-time heatmap using Plotly
fig = px.scatter_3d(df, x='lon', y='lat', z='cell_t', color='ship_count')
fig.update_traces(marker=dict(size=2))  # Set marker size to 2 (or smaller as needed)
fig.update_layout(scene=dict(zaxis_title='Time', xaxis_title='Longitude', yaxis_title='Latitude'))
fig.show()


# 4. Time Slider with 2D Heatmap

Finally, we’ll create a 2D heatmap with a time slider to visualize the spatial distribution of ship counts at different times.

In [ ]:
import psycopg as pg
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime
import numpy as np

# Connect to the PostgreSQL/MobilityDB database
connection = pg.connect(
    host="localhost",  # or your database host
    port="5432",
    dbname="mobility_analytics_chapter",
    user="postgres",
    password="password"  # Replace with your actual password
)

# Query to fetch spatial heatmap data with timestamps
query = """
    SELECT ST_X(ST_TRANSFORM(cell_geom, 4326)) AS lon, 
        ST_Y(ST_TRANSFORM(cell_geom, 4326)) AS lat, 
        COUNT(*) AS ship_count, cell_t
    FROM tHeatmap
    GROUP BY cell_geom, cell_t;
"""
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

# Apply log transformation to the 'count' column to better visualize the data
df['log_count'] = df['ship_count'].apply(lambda x: np.log(x + 1))  # Adding 1 to avoid log(0)

# Ensure the time column is correctly formatted as string
df['cell_t'] = df['cell_t'].dt.strftime('%Y-%m-%d %H:%M:%S')  # Format timestamp to a string

# Group data by time and prepare it for HeatMapWithTime
time_indexed_data = []
time_steps = sorted(df['cell_t'].unique())  # Unique time steps sorted
for time in time_steps:
    # Extract data for this specific time step
    subset = df[df['cell_t'] == time]
    heat_data = [[row['lat'], row['lon'], row['log_count']] for index, row in subset.iterrows()]
    time_indexed_data.append(heat_data)

# Create a base map centered on Denmark
m = folium.Map(location=[55.6761, 12.5683], zoom_start=7, tiles="cartodb positron")

# Add HeatMap with TimeSlider functionality
HeatMapWithTime(
    time_indexed_data,
    index= time_steps,
    gradient={0.2: 'white', 0.6: 'orange', 1.0: 'red'}  # Custom color gradient
).add_to(m)

# Save and display the map
m.save('TimeSlideHeatmap.html')
m


# 5. Flow maps


In [ ]:
import psycopg as pg
import pandas as pd
import folium
from folium.plugins import AntPath
import random

# Connect to PostgreSQL database
connection = pg.connect(
    host="localhost",  # or your database host
    port="5432",
    dbname="mobility_analytics_chapter",
    user="postgres",
    password="password"  # Replace with your actual password
)

# Query to retrieve distinct ports and port-to-port flow data
query_ports = """
    SELECT id, ST_X(ST_TRANSFORM(ST_Centroid(geom), 4326)) as lon, 
           ST_Y(ST_TRANSFORM(ST_Centroid(geom), 4326)) as lat
    FROM harbours
    WHERE id IN (
        SELECT porta FROM flow
        UNION
        SELECT portb FROM flow
    );
"""

query_flows = """
    SELECT a.id as porta, b.id as portb, count(distinct mmsi) as ship_count,
           ST_X(ST_TRANSFORM(ST_Centroid(a.geom), 4326)) as lon_a, ST_Y(ST_TRANSFORM(ST_Centroid(a.geom), 4326)) as lat_a,
           ST_X(ST_TRANSFORM(ST_Centroid(b.geom), 4326)) as lon_b, ST_Y(ST_TRANSFORM(ST_Centroid(b.geom), 4326)) as lat_b
    FROM harbours a
    JOIN harbours b ON a.id > b.id
    JOIN flow f ON a.id=f.porta AND b.id=f.portb
    GROUP BY a.id, b.id, lon_a, lat_a, lon_b, lat_b;
"""

# Fetch the ports and flow data
df_ports = pd.read_sql_query(query_ports, connection)
df_flows = pd.read_sql_query(query_flows, connection)

# Close the connection
connection.close()

# Create a base map
m = folium.Map(location=[56.2639, 9.5018], zoom_start=6, tiles="cartodb positron")  # Centered around Denmark

buffer_radius = 2000  # Adjust the buffer size as necessary

# Function to generate a random color in hex format
def get_random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Add distinct ports as markers
for index, row in df_ports.iterrows():
    folium.Circle(
        location=[row['lat'], row['lon']],
        radius=buffer_radius,
        color=get_random_color(),
        fill=True,
        fill_color=get_random_color(),
        fill_opacity=0.6
    ).add_to(m)

# Add flow arrows between ports, with size based on ship count
for index, row in df_flows.iterrows():
    # Calculate arrow thickness based on the number of ships
    line_weight = max(1, row['ship_count'] / 10.0)  # Scale down the weight
    
    # Add an arrow between the two ports
    AntPath(
        locations=[(row['lat_a'], row['lon_a']), (row['lat_b'], row['lon_b'])],
        color='blue',
        weight=line_weight*5,  # Thickness represents ship flow
        opacity=0.6
    ).add_to(m)

# Save the map to an HTML file and display it
m.save('FlowMap.html')
m
